In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-03"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1449,2024-07-03,Mundo Amistosos Internacionais,06:30,Austrália F,China F,1.17,4.56,143.5,1.88,1.78,0.0,0.00,0.00,W8zxmSVg,0.854701,0.219298,0.531915,0.561798,0.073999,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,85,56,2.65,2.93,0.000,0.000,0.836681,0.038640,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1450,2024-07-03,Mundo Amistosos Internacionais,13:00,Eslováquia U20 F,Áustria U20 F,1.13,5.40,122.5,1.82,1.88,0.0,0.00,0.00,0lShqpVI,0.884956,0.185185,0.549451,0.531915,0.070141,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.924761,0.022933,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1451,2024-07-03,Nova Zelândia Nbl,04:30,Wellington Saints,Whai,1.04,8.90,180.5,1.85,1.85,0.0,0.00,0.00,fuBfE9dS,0.961538,0.112360,0.540541,0.540541,0.073898,184.894,70.930296,0.383627,1.8,1.643168,0.912871,114.48,1.798,0.811862,0.451536,52.0,334.878,287.578339,0.858756,1.2,1.643168,1.369306,511.56,4.196,3.633769,0.866008,-40.0,106,87,1.08,5.88,123.574,400.044,1.118282,0.000000,NaN,-2.82,-0.564,-0.070922,0.000000,0.0,0.000000,-3.25,-0.650,-12.153846,0.000000,0.0,0.000000
1452,2024-07-03,Singapura Nbl,08:00,Nanyang,Siglap,3.04,1.35,125.5,1.85,1.85,0.0,0.00,0.00,reGYHeWj,0.328947,0.740741,0.540541,0.540541,0.069688,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.544424,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1453,2024-07-03,Venezuela Superliga,20:00,Gladiadores,Llaneros,1.12,5.72,160.5,1.86,1.84,0.0,0.00,0.00,2Far1GfK,0.892857,0.174825,0.537634,0.543478,0.067682,94.354,9.901120,0.104936,2.4,1.341641,0.559017,107.36,1.136,0.065422,0.057590,35.0,200.980,50.630268,0.251917,1.2,1.643168,1.369306,244.94,2.724,0.720472,0.264490,-17.0,88,74,1.22,3.31,105.436,270.110,0.951079,0.007644,NaN,-0.21,-0.042,-2.857143,0.000000,0.0,0.000000,-5.00,-1.000,-4.720000,0.000000,0.0,0.000000
1454,2024-07-03,Mundo Jogos Olímpicos,18:30,Lituânia,Costa do Marfim,1.02,12.60,161.5,1.80,1.92,0.0,0.00,0.00,UDcAZMNQ,0.980392,0.079365,0.555556,0.520833,0.059757,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,96,0,1.01,0.00,0.000,0.000,1.202393,0.045620,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1455,2024-07-03,Vietnã Vba,09:30,Can Tho Catfish,Hanoi Buffaloes,4.75,1.16,184.5,1.80,1.86,0.0,0.00,0.00,j1TpJtoS,0.210526,0.862069,0.555556,0.537634,0.072595,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,84,0.00,2.97,0.000,0.000,0.859057,0.023184,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1456,2024-07-03,Mundo Jogos Olímpicos,11:30,Croácia,Nova Zelândia,1.07,7.50,168.5,1.86,1.88,0.0,0.00,0.00,pQN6F1Wf,0.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
1458,21:30,Colômbia Lbp,Titanes de Barranquilla,Bucaros,1.54,Back Home
